In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/02 16:18:07 WARN Utils: Your hostname, Desktop-Gar resolves to a loopback address: 127.0.1.1; using 172.25.243.204 instead (on interface eth0)
24/03/02 16:18:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 16:18:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/02 16:18:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.createOrReplaceTempView ('green')

In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    ROUND(SUM(total_amount), 2) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

24/03/02 16:19:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 16:19:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [6]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

/mnt/d/apache/spark-3.4.2-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [7]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    ROUND(SUM(total_amount), 2) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

24/03/02 16:19:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [9]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [10]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [11]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [12]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [13]:
df_join = spark.read.parquet('data/report/revenue/total')

In [14]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [15]:
df_zones = spark.read.parquet('zones/')

In [16]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [17]:
revenue_zones = df_result.drop('LocationID', 'zone').fillna(0)

In [18]:
revenue_zones.show()

+-------------------+------------+--------------------+-------------+---------------------+---------+------------+
|               hour|green_amount|green_number_records|yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------+--------------------+-------------+---------------------+---------+------------+
|2020-01-01 00:00:00|         0.0|                   0|        42.41|                    2|   Queens|   Boro Zone|
|2020-01-01 00:00:00|         0.0|                   0|          8.8|                    1| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|         0.0|                   0|        34.09|                    1|   Queens|   Boro Zone|
|2020-01-01 00:00:00|      195.03|                   9|       220.21|                    8| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|        87.6|                   3|       754.95|                   45|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|        61.3|                   1|          0.0|           

In [19]:
df_result.drop('LocationID', 'zone').write.parquet('data/report/revenue/revenue-zones', mode='overwrite')

In [20]:
!ls -la data/report/revenue/revenue-zones

total 1260
drwxrwxrwx 1 root root   4096 Mar  2 16:20 .
drwxrwxrwx 1 root root   4096 Mar  2 16:20 ..
-rwxrwxrwx 1 root root      8 Mar  2 16:20 ._SUCCESS.crc
-rwxrwxrwx 1 root root   2484 Mar  2 16:20 .part-00000-8b9b3bbd-e3f4-469b-b84e-279ee72169f8-c000.snappy.parquet.crc
-rwxrwxrwx 1 root root   2480 Mar  2 16:20 .part-00001-8b9b3bbd-e3f4-469b-b84e-279ee72169f8-c000.snappy.parquet.crc
-rwxrwxrwx 1 root root   2476 Mar  2 16:20 .part-00002-8b9b3bbd-e3f4-469b-b84e-279ee72169f8-c000.snappy.parquet.crc
-rwxrwxrwx 1 root root   2452 Mar  2 16:20 .part-00003-8b9b3bbd-e3f4-469b-b84e-279ee72169f8-c000.snappy.parquet.crc
-rwxrwxrwx 1 root root      0 Mar  2 16:20 _SUCCESS
-rwxrwxrwx 1 root root 316513 Mar  2 16:20 part-00000-8b9b3bbd-e3f4-469b-b84e-279ee72169f8-c000.snappy.parquet
-rwxrwxrwx 1 root root 316124 Mar  2 16:20 part-00001-8b9b3bbd-e3f4-469b-b84e-279ee72169f8-c000.snappy.parquet
-rwxrwxrwx 1 root root 315898 Mar  2 16:20 part-00002-8b9b3bbd-e3f4-469b-b84e-279ee72169f8-c000.snappy.